In [58]:
import pandas as pd
import numpy as np

In [59]:
transcriptsv1 = pd.read_csv("transcripts_dataset_v1.csv.gz", compression="gzip")
transcriptsv2 = pd.read_csv("transcripts_dataset_v2.csv.gz", compression="gzip")
metadata = pd.read_csv("metadata.tsv", sep= "\t")

In [61]:
# print('Transcipt v1 shape:', transcriptsv1.shape)
# print('Transcipt v2 shape:', transcriptsv2.shape)
# print('Metadata shape:', metadata.shape, '\n')

# print(list(transcriptsv1.columns))
# print(list(transcriptsv2.columns))
# print(list(metadata.columns))

# print('\n')

# print('Transcript v1 null check:\n', transcriptsv1.isna().sum(), '\n')
# print('Transcript v2 null check:\n', transcriptsv2.isna().sum(), '\n')
print('Metadata null check:\n', metadata.isna().sum()) # show_name 2 nulls, episode_description 205 nulls

Metadata null check:
 show_uri                     0
show_name                    0
show_description             2
publisher                    0
language                     0
rss_link                     0
episode_uri                  0
episode_name                 0
episode_description        205
duration                     0
show_filename_prefix         0
episode_filename_prefix      0
dtype: int64


In [62]:
# concatenate transcripts dataframes
transcripts = pd.concat([transcriptsv1, transcriptsv2])
print(transcripts.shape)

# subset metadata
metadata = metadata[["show_name", "show_description", "publisher", "language", "episode_name", "episode_description", "duration", "show_filename_prefix", "episode_filename_prefix"]]
metadata = metadata.rename({"episode_filename_prefix": "episode_id", "show_filename_prefix": "show_id"}, axis="columns")  # rename cols

# remove episode_id suffix 
transcripts.episode_id = transcripts.episode_id.apply(lambda x: x.replace(".json", ""))
transcripts = transcripts.drop("Unnamed: 0", axis=1)

# remove whitespace
metadata.episode_id = metadata.episode_id.apply(lambda x: x.strip())

(105360, 6)


In [66]:
# join data
full_dataset = transcripts.join(metadata.set_index("episode_id"), on="episode_id", rsuffix="_trans")
full_dataset.isna().sum()

full_dataset.shape
full_dataset.head(5)

(105360, 13)

In [2]:
# Null inspection
# full_dataset[full_dataset.isnull().any(axis=1)] # metadata discrepancies, episode_id present for all instances

In [67]:
# Save the DataFrame as a gzip-compressed CSV file
full_dataset.to_csv('transcripts_dataset_final.csv.gz', compression='gzip', index=False)

## Create dataset with 200 word documents (on subset)

In [1]:
import pandas as pd
t_data_sample = pd.read_csv("transcripts_sample.csv.gz", compression="gzip")